In [1]:
# Standard data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Web scraping libraries
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
import requests
import bs4

from bs4 import BeautifulSoup
import re
import os
from os.path import basename

# Some styling
%matplotlib inline
plt.style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = (14, 8)
plt.rc('axes', titlesize=22) 
plt.rc('figure', titlesize=22)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=14)


In [2]:
URL_ALL_WATCHES = r"https://www.watchfinder.co.uk/all-watches?orderby=BestMatch&pageno="
URL_BASE = r"https://www.watchfinder.co.uk"

In [3]:
keys = ['id',
        'brand',
        'series',
        'model',
        'price',
        'Year',
        'Box',
        'Papers',
        'Manufacturer warranty',
        'Watchfinder warranty',
        'Availability',
        'Delivery',
        'Returns policy',
        'Product code',
        'Movement',
        'Case size',
        'Case material',
        'Bracelet material',
        'Dial type',
        'Water resistance',
        'Location',
        'img_name']

In [4]:
def g(i):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(URL_ALL_WATCHES + i, headers=headers)
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    a_tags = bs.find_all('a', {'class': 'prods_name redirect'})
    
    urls = []
    
    for i in range(len(a_tags)):
        urls.append(a_tags[i].attrs['href'])
        
    for i in range(len(urls)):
        urls[i] = urls[i].replace(' ', '%20')
    
    return urls

In [5]:
%%time

urls = []
for i in range(1, 115):
    urls = urls + g(str(i))

Wall time: 3min 51s


In [6]:
for i in range(len(urls)):
    if urls[i].find('é') > 1:
        urls[i] = urls[i].replace('é', '\\xe9')

In [7]:
def f(url, idx):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url, headers=headers)
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    table_data = bs.find_all('tr')
    
    watches_dict = {}

    for key in keys:
        watches_dict[key] = None
    
    watches_dict['id'] = idx
    
    for i in range(len(table_data)):

        if len(list(table_data[i])) == 5:

            if list(table_data[i])[1].find('div'):
                key = list(list(table_data[i])[1].find('div'))[0]            
                value = list(list(table_data[i])[3])[0]

                watches_dict[key] = value

            else:
                key = list(list(table_data[i])[1])[0]            
                value = list(list(table_data[i])[3])[0]

                watches_dict[key] = value
                
    brand = list(bs.find('span', {'class': 'prod_brand ellipsis'}))[0]
    series = list(bs.find('span', {'class': 'prod_series ellipsis'}))[0]
    model = list(bs.find('span', {'class': 'prod_model ellipsis'}))[0]
    price = bs.find('meta', {'itemprop': 'price'}).attrs['content']
    
    watches_dict['brand'] = brand
    watches_dict['series'] = series
    watches_dict['model'] = model
    watches_dict['price'] = price
    
    try:
        image = bs.find('meta', {'itemprop': 'image'})
        img_url = image.attrs['content']
        img_name = idx + '.jpg'
        watches_dict['img_name'] = img_name
        save_path = os.path.join('./images/', img_name)
        with open(save_path, 'wb') as handle:
                response = requests.get(img_url, stream=True)

                if not response.ok:
                    print(response)

                for block in response.iter_content(1024):
                    if not block:
                        break

                    handle.write(block)
        
    except:
        pass
        
    return watches_dict

In [14]:
%%time

dicts = []

for idx in range(len(urls)):
    
    try:
        dicts.append(f(URL_BASE + urls[i], idx))
        
    except Exception as e:
        print(url)
        print(e)

/Omega/Planet%20Ocean/232.32.46.21.01.001/23749/item/183674?searchId=2c6e0d14-7137-400a-b53d-6a6b63cab628&rank=15
'NoneType' object has no attribute 'attrs'
/Panerai/Submersible/PAM00974/42374/item/182966?searchId=79cb6c9c-7777-4a17-9926-f2309aaee802&rank=20
'NoneType' object has no attribute 'attrs'
/Rolex/Air-King/116900/29608/item/184106?searchId=74e7653c-7f1b-405d-aa3a-6e4f5bb38440&rank=26
'NoneType' object has no attribute 'attrs'
<Response [404]>
/Rolex/Oysterquartz%20Datejust/17000/34171/item/179826?searchId=26527d5a-3b17-41d4-85a1-bdbb51e2c3e4&rank=20
'NoneType' object has no attribute 'attrs'
/Panerai/Luminor%20Submersible/PAM00615/27693/item/156423?searchId=b6270c32-2baa-4e06-af10-a51a15126aae&rank=1
'NoneType' object has no attribute 'attrs'
/Cartier/Calibre%20de%20Cartier/W7100036/17882/item/157760?searchId=d0d94394-a0a1-46b4-9867-729a464a9c85&rank=18
'NoneType' object has no attribute 'attrs'
Wall time: 1h 13min 10s


In [15]:
big_dict = {}

for k in dicts[0]:
    big_dict[k] = [d[k] for d in dicts]

In [16]:
df = pd.DataFrame.from_dict(big_dict)

In [17]:
df.to_csv('wf_df_raw.csv')

In [ ]:
# df = pd.read_csv('wf_df_raw.csv')